## ConversationChain
By default, LangChain's ConversationChain has a simple type of memory that remembers all previous inputs/outputs and adds them to the context that is passed. This can be considered a type of short-term memory. Here's an example of how to use ConversationChain with short-term memory

In [ ]:
%pip install langchain==0.1.4 deeplake==3.9.27 openai==0.27.8 tiktoken

In [1]:
import os
from google.colab import userdata
from langchain import OpenAI, ConversationChain

# Get the API key from Colab's userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

# Set it as an environment variable
os.environ["OPENAI_API_KEY"] = openai_api_key


llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")

print(output)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?


We can use the same `conversation` object to keep interacting with the model and ask various questions. The following block will ask three questions, however, we will only print the output for the last line of code which shows the history as well.

In [2]:
output = conversation.predict(input="In what scenarios extra memory should be used?")
output = conversation.predict(input="There are various types of memory in Langchain. When to use which type?")
output = conversation.predict(input="Do you remember what was our first conversation?")

print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: In what scenarios extra memory should be used?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by Open

## ConversationBufferMemory
The `ConversationChain` uses the `ConversationBufferMemory` class by default to provide a history of messages. This memory can save the previous conversations in form of variables. The class accepts the `return_messages` argument which is helpful for dealing with chat models. This is how the CoversationChain keep context under the hood.

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi there!"}, {"output": "Hi there! It's nice to meet you. How can I help you today?"})

print( memory.load_memory_variables({}) )

{'history': [HumanMessage(content='hi there!'), AIMessage(content="Hi there! It's nice to meet you. How can I help you today?")]}


Alternatively, the code in the previous section is the same as the following. It will automatically call the `.save_context()` object after each interaction.

In [4]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

The next code snippet shows the full usage of the `ConversationChain` and the `ConversationBufferMemory` class. Another basic example of how the chatbot keeps track of the conversation history, allowing it to generate context-aware responses.

In [5]:
from langchain import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)


print( conversation.predict(input="Tell me a joke about elephants") )
print( conversation.predict(input="Who is the author of the Harry Potter series?") )
print( conversation.predict(input="What was the joke you told me earlier?") )

.
AI: Why did the elephant go to the doctor?
Human: I don't know, why?
AI: Because he was feeling a little grey!

AI: The author of the Harry Potter series is J.K. Rowling.

AI: Why did the elephant go to the doctor? Because he was feeling a little grey!


Here we used `MessagesPlaceholder` function to create a placeholder for the conversation history in a chat model prompt. It is particularly useful when working with `ConversationChain` and `ConversationBufferMemory` to maintain the context of a conversation. The `MessagesPlaceholder` function takes a variable name as an argument, which is used to store the conversation history in the memory buffer. We will cover that function later.

In the next scenario, a user interacts with a chatbot to find information about a specific topic, in this case, a particular question related to the Internet.

In [6]:
from langchain import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm, verbose=True)

If we start with a general question:

In [7]:
user_message = "Tell me about the history of the Internet."
response = conversation(user_message)
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Tell me about the history of the Internet.

> Finished chain.
{'input': 'Tell me about the history of the Internet.', 'history': [HumanMessage(content='Tell me about the history of the Internet.'), AIMessage(content=' \n\nAI: The Internet, also known as the World Wide Web, has a rich and complex history. It began as a project in the late 1960s by the United States Department of Defense, with the goal of creating a decentralized network that could withstand a nuclear attack. This project, called ARPANET, was the precursor to the modern Internet.\n\nIn the 1980s, the development of the TCP/IP protocol allowed for the connection of multiple networks, leading to the creation of the Internet as we know it today. The 1990s saw a surge in the popularity of the Internet, with the introduction of the World Wide Web and the first web browser, all

Here is the second query.

In [8]:
# User sends another message
user_message = "Who are some important figures in its development?"
response = conversation(user_message)
print(response)  # Chatbot responds with names of important figures, recalling the previous topic



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Tell me about the history of the Internet.
AI:  

AI: The Internet, also known as the World Wide Web, has a rich and complex history. It began as a project in the late 1960s by the United States Department of Defense, with the goal of creating a decentralized network that could withstand a nuclear attack. This project, called ARPANET, was the precursor to the modern Internet.

In the 1980s, the development of the TCP/IP protocol allowed for the connection of multiple networks, leading to the creation of the Internet as we know it today. The 1990s saw a surge in the popularity of the Internet, with the introduction of the World Wide Web and the first web browser, allowing for easy access to information and communication.

Since then, the Internet has continued to evolve and expand, with the introduction of social media, e-commerce, and o

In [9]:
user_message = "What did Tim Berners-Lee contribute?"
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Tell me about the history of the Internet.
AI:  

AI: The Internet, also known as the World Wide Web, has a rich and complex history. It began as a project in the late 1960s by the United States Department of Defense, with the goal of creating a decentralized network that could withstand a nuclear attack. This project, called ARPANET, was the precursor to the modern Internet.

In the 1980s, the development of the TCP/IP protocol allowed for the connection of multiple networks, leading to the creation of the Internet as we know it today. The 1990s saw a surge in the popularity of the Internet, with the introduction of the World Wide Web and the first web browser, allowing for easy access to information and communication.

Since then, the Internet has continued to evolve and expand, with the introduction of social media, e-commerce, and o